<a href="https://colab.research.google.com/github/tmaiecnj/CPE-313/blob/main/Hands-on%20Activity%207.1%20-%20Deep%20Learning%20in%20Time%20Series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Github Link: [Hands-on Activity 7.1: Deep Learning in Time Series]()

# Activity 4.4 Deep Learning in Time Series

## Objective(s):
This activity aims to introduce how to build and train a simple RNN or LSTM for time series forecasting, using Keras.

## Intended Learning Outcomes (ILOs):
* Demonstrate how to build and train RNN for time series forecasting
* Demonstrate how to build and train LSTM for time series forecasting

## Resources:
* Jupyter Notebook
* Beijing.csv

## Procedures

Load the necessary libraries

In [ ]:
%matplotlib inline

import sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd

import keras
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM

Build a recurrent neural network and train it to forecast a single time series.
* Load and setup the dataset

In [ ]:
df_Beijing = pd.read_csv('Beijing.csv')
df_Beijing = df_Beijing[df_Beijing.year >= 2015]
df_Beijing.head(10)

Forecast the 'PM' series, which are measurements of air pollution for several different districts. Note that there are occasional missing values in these series, which we can fill with simple linear interpolation. To start, we'll focus on the "PM_Dongsi" series and interpolate the missing values.

In [ ]:
df_Beijing['PM_Dongsi'] = df_Beijing['PM_Dongsi'].interpolate()
df_Beijing['PM_Dongsi'].head(10)

Create a function to extract and plot the last *n* days of data

In [ ]:
def get_n_last_days(df, series_name, n_days):
  """
  Extract last n_days of an hourly time series
  """
    return df[series_name][-(24*n_days):]

In [ ]:
def plot_n_last_days(df, series_name, n_days):
  """
  Plot last n_days of an hourly time series
  """
  plt.plot(get_n_last_days(df, series_name, n_days), 'k-')
  plt.title('{0} Air Quality Time Series - {1} days'.format(series_name, n_days))

  plt.xlabel('Recorded Hour')
  plt.ylabel('Reading')
  plt.grid(alpha=0.3)

Check last 6 weeks of data look like

In [ ]:
plot_n_last_days(df_Beijing, 'PM_Dongsi', 42)

Train a simple RNN to forecast the PM_Dongsi time series

In [ ]:
def get_keras_format_series(series):
  """
  Convert a series to a numpy array of shape
  [n_samples, time_steps, features]
  """

  series = np.array(series)
  return series.reshape(series.shape[0], series.shape[1], 1)

In [ ]:
def get_train_test_data(df, series_name, series_days, input_hours, test_hours, sample_gap=3):
  """
  Utility processing function that splits an hourly time series into
  train and test with keras-friendly format, according to user-specified
  choice of shape.

  arguments
  ---------
  df (dataframe): dataframe with time series columns
  series_name (string): column name in df
  series_days (int): total days to extract
  input_hours (int): length of sequence input to network
  test_hours (int): length of held-out terminal sequence
  sample_gap (int): step size between start of train sequences; default 5

  returns
  ---------
  tuple: train_X, test_X_init, train_y, test_y
  """

  forecast_series = get_n_last_days(df, series_name, series_days).values

  train = forecast_series[:-test_hours]
  test = forecast_series[-test_hours:]

  train_X, train_y = [], []

  for i in range(0, train.shape[0]-input_hours, sample_gap):
    train_X.append(train[i:i+input_hours])
    train_y.append(train[i+input_hours])

  train_X = get_keras_format_series(train_X)
  train_y = np.array(train_y)

  test_X_init = test[:input_hours]
  test_y = test[input_hours:]

  return train_X, test_X_init, train_y, test_y

With the get_train_test_data utility function in hand, we're all set to extract keras-friendly arrays and
start training simple RNN models. We run this function in the cell below. We use the last 56 days of
the PM_Dongsi series, and will train a model that takes in 12 time steps in order to predict the next
time step. We use the last day of data for visually testing the model.

In [ ]:
series_days = 56
input_hours = 12
test_hours = 24

In [ ]:
train_X, test_X_init, train_y, test_y = (get_train_test_data(df_Beijing,
                                                             'PM_Dongsi',
                                                             series_days,
                                                             input_hours,
                                                             test_hours))

Below we see that by taking multiple time slices, we get 436 training samples of 12 time steps each.

In [ ]:
print('Training shape: {}'.format(train_X.shape))

Train a simple RNN model using keras.

In [ ]:
def fit_SimpleRNN(train_X, train_y, cell_units, epochs):
  """
  Fit Simple RNN to data train_X, train_y

  arguments
  ---------
  train_X (array): input sequence samples for training
  train_y (list): next step in sequence targets
  cell_units (int): number of hidden units for RNN cells
  epochs (int): number of training epochs
  """

  # initialize model
  model = Sequential()

  # construct an RNN layer with specified number of hidden units
  # per cell and desired sequence input format
  model.add(SimpleRNN(cell_units, input_shape=(train_X.shape[1],1)))

  # add an output layer to make final predictions
  model.add(Dense(1))

  # define the loss function / optimization strategy, and fit
  # the model with the desired number of passes over the data (epochs)
  model.compile(loss='mean_squared_error', optimizer='adam')
  model.fit(train_X, train_y, epochs=epochs, batch_size=64, verbose=0)

  return model

Use this function to fit a very simple baseline model.

In [ ]:
model = fit_SimpleRNN(train_X, train_y, cell_units=10, epochs=10)

This prediction method and a utility function for plotting its output against the ground truth are
defined below.

In [ ]:
def predict(X_init, n_steps, model):
  """
  Given an input series matching the model's expected format,
  generates model's predictions for next n_steps in the series
  """

  X_init = X_init.copy().reshape(1,-1,1)
  preds = []

  # iteratively take current input sequence, generate next step pred,
  # and shift input sequence forward by a step (to end with latest pred).
  # collect preds as we go.
  for _ in range(n_steps):
    pred = model.predict(X_init)
    preds.append(pred)
    X_init[:,:-1,:] = X_init[:,1:,:]
    X_init[:,-1,:] = pred

  preds = np.array(preds).reshape(-1,1)
  return preds

In [ ]:
def predict_and_plot(X_init, y, model, title):
  """
  Given an input series matching the model's expected format,
  generates model's predictions for next n_steps in the series,
  and plots these predictions against the ground truth for those steps

  arguments
  ---------
  X_init (array): initial sequence, must match model's input shape
  y (array): true sequence values to predict, follow X_init
  model (keras.models.Sequential): trained neural network
  title (string): plot title
  """

  y_preds = predict(test_X_init, n_steps=len(y), model=model)

  start_range = range(1, input_hours+1)
  predict_range = range(input_hours, test_hours)

  plt.plot(start_range, test_X_init)
  plt.plot(predict_range, test_y, color='orange')
  plt.plot(predict_range, y_preds, color='teal', linestyle='--')

  plt.title(title)
  plt.legend(['Initial Series','Target Series','Predictions'])

In [ ]:
predict_and_plot(test_X_init,
                 test_y, model,
                 'PM Series: Test Data and Simple RNN Predictions')

Retrain the model a few times in order to get results that we're happy with.

In [ ]:
model = fit_SimpleRNN(train_X, train_y, cell_units=30, epochs=1200)

In [ ]:
predict_and_plot(test_X_init,
                 test_y, model,
                 'PM Series: Test Data and Simple RNN Predictions')

In [ ]:
model.summary()

### LSTM

Write a new function for fitting an LSTM with keras

In [ ]:
def fit_LSTM(train_X, train_y, cell_units, epochs):
  """
  Fit LSTM to data train_X, train_y

  arguments
  ---------
  train_X (array): input sequence samples for training
  train_y (list): next step in sequence targets
  cell_units (int): number of hidden units for LSTM cells
  epochs (int): number of training epochs
  """

  # initialize model
  model = Sequential()

  # construct a LSTM layer with specified number of hidden units
  # per cell and desired sequence input format
  model.add(LSTM(cell_units, input_shape=(train_X.shape[1],1)))

  # add an output layer to make final predictions
  model.add(Dense(1))

  # define the loss function / optimization strategy, and fit
  # the model with the desired number of passes over the data (epochs)
  model.compile(loss='mean_squared_error', optimizer='adam')
  model.fit(train_X, train_y, epochs=epochs, batch_size=64, verbose=0)

  return model

Train a LSTM to forecast the PM_Nongzhanguan time series

In [ ]:
series_days = 50
input_hours = 12
test_hours = 96

In [ ]:
train_X, test_X_init, train_y, test_y = (get_train_test_data(df_Beijing,
                                                             'PM_Nongzhanguan',
                                                             series_days,
                                                             input_hours,
                                                             test_hours))

In [ ]:
model = fit_LSTM(train_X, train_y, cell_units=70, epochs=3000)

In [ ]:
predict_and_plot(test_X_init,
                 test_y, model,
                 'PM_Nongzhanguan Series: Test Data and LSTM Predictions')

In [ ]:
model.summary()

## Supplementary Activity

* Train your own model to forecast the "PM_Nongzhanguan" series from the Beijing dataframe.
* Train a LSTM to forecast the PM_Dongsi time series

## Conclusion

*Conclusion*